In [1]:
import os
import json
import numpy as np
import pandas as pd
from shapely.geometry import Polygon
from shapely.validation import explain_validity

def check_and_fix_polygon(points):
    """Memeriksa dan memperbaiki polygon jika tidak valid."""
    polygon = Polygon(points)

    if not polygon.is_valid:
        print("Polygon tidak valid! Kesalahan:", explain_validity(polygon))

        # 1. Perbaiki dengan buffer(0)
        fixed_polygon = polygon.buffer(0)
        if fixed_polygon.is_valid:
            print("Polygon berhasil diperbaiki dengan buffer(0).")
            return fixed_polygon
        
        # 2. Pastikan polygon tertutup (titik awal = titik akhir)
        if points[0] != points[-1]:
            points.append(points[0])
            polygon = Polygon(points)
            if polygon.is_valid:
                print("Polygon berhasil diperbaiki dengan menutup titik.")
                return polygon

        # 3. Hapus titik duplikat
        unique_points = []
        for p in points:
            if p not in unique_points:
                unique_points.append(p)
        polygon = Polygon(unique_points)
        if polygon.is_valid:
            print("Polygon berhasil diperbaiki dengan menghapus titik duplikat.")
            return polygon

        # 4. Gunakan convex hull sebagai solusi terakhir
        fixed_polygon = polygon.convex_hull
        print("Menggunakan convex hull sebagai solusi terakhir.")
        return fixed_polygon
    
    return polygon

def compute_iou(poly1, poly2):
    """Menghitung Intersection over Union (IoU) antara dua polygon."""
    intersection = poly1.intersection(poly2).area
    union = poly1.union(poly2).area
    return intersection / union if union > 0 else 0.0

def process_json_files(gt_folder, pred_folder, output_file):
    """Membaca file JSON dari dua folder dan menyimpan IoU ke dalam file Excel."""
    results = []

    for filename in os.listdir(gt_folder):
        if filename.endswith(".json") and filename in os.listdir(pred_folder):
            gt_filepath = os.path.join(gt_folder, filename)
            pred_filepath = os.path.join(pred_folder, filename)

            with open(gt_filepath, 'r') as gt_file, open(pred_filepath, 'r') as pred_file:
                gt_data = json.load(gt_file)
                pred_data = json.load(pred_file)

                if "shapes" in gt_data and "shapes" in pred_data:
                    gt_polygons = {shape["label"].strip().lower(): shape["points"] for shape in gt_data["shapes"]}
                    pred_polygons = {shape["label"].strip().lower(): shape["points"] for shape in pred_data["shapes"]}

                    # Debugging: Periksa label yang tersedia
                    print(f"\n=== {filename} ===")
                    print("Ground Truth labels:", gt_polygons.keys())
                    print("Prediction labels:", pred_polygons.keys())

                    # Cek semua label yang ada di ground truth dan prediction
                    common_labels = set(gt_polygons.keys()).intersection(set(pred_polygons.keys()))

                    for label in common_labels:
                        gt_polygon = check_and_fix_polygon(gt_polygons[label])
                        pred_polygon = check_and_fix_polygon(pred_polygons[label])

                        if gt_polygon.is_valid and pred_polygon.is_valid:
                            iou = compute_iou(gt_polygon, pred_polygon)
                            results.append({"Image": filename, "Object": label, "IoU": iou})
                            print(f"IoU untuk {label} di {filename} = {iou:.4f}")
                        else:
                            print(f"Polygon untuk {label} tetap tidak valid setelah perbaikan, IoU tidak dihitung!")

    # Simpan hasil ke file Excel
    df = pd.DataFrame(results)
    df.to_excel(output_file, index=False)
    print(f"Hasil IoU disimpan di {output_file}")

# Contoh penggunaan
gt_folder = "D:\\DATA RAFKI\\PunyaResti\\PunyaResti\\DATASET SMARTPHONE\\4CH\\test\\images"  # Ganti dengan path folder ground truth
pred_folder = "D:\\DATA RAFKI\\PunyaResti\\PunyaResti\\DATASET SMARTPHONE\\4CH\\TEST_4CH_JSON"  # Ganti dengan path folder prediction
output_file = "D:\\DATA RAFKI\\PunyaResti\\PunyaResti\\DATASET SMARTPHONE\\4CH\\TEST_4CH.xlsx"  # File output Excel
process_json_files(gt_folder, pred_folder, output_file)


=== ABBAS-MURTHADO_NORMAL_4CH_frame_124.json ===
Ground Truth labels: dict_keys(['ivs', 'ias'])
Prediction labels: dict_keys(['ivs', 'ias'])
IoU untuk ivs di ABBAS-MURTHADO_NORMAL_4CH_frame_124.json = 0.6506
IoU untuk ias di ABBAS-MURTHADO_NORMAL_4CH_frame_124.json = 0.5477

=== ABBAS-MURTHADO_NORMAL_4CH_frame_161.json ===
Ground Truth labels: dict_keys(['ivs', 'ias'])
Prediction labels: dict_keys(['ias', 'ivs'])
IoU untuk ias di ABBAS-MURTHADO_NORMAL_4CH_frame_161.json = 0.6896
IoU untuk ivs di ABBAS-MURTHADO_NORMAL_4CH_frame_161.json = 0.5425

=== ABBAS-MURTHADO_NORMAL_4CH_frame_17.json ===
Ground Truth labels: dict_keys(['ivs', 'ias'])
Prediction labels: dict_keys(['ias', 'ivs'])
IoU untuk ias di ABBAS-MURTHADO_NORMAL_4CH_frame_17.json = 0.7248
IoU untuk ivs di ABBAS-MURTHADO_NORMAL_4CH_frame_17.json = 0.6779

=== ABBAS-MURTHADO_NORMAL_4CH_frame_186.json ===
Ground Truth labels: dict_keys(['ivs', 'ias'])
Prediction labels: dict_keys(['ias', 'ivs'])
IoU untuk ias di ABBAS-MURTHADO_N

In [ ]:
pip install shapely
pip install openpyxl

   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------------------ --------------------- 0.8/1.7 MB 6.7 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 7.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install openpyxl

  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached et_xmlfile-2.0.0-py3-none-any.whl (18 kB)
Note: you may need to restart the kernel to use updated packages.
